In [86]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [87]:
data = pd.read_csv('train.csv')

In [88]:
data.shape

(1460, 63)

# Constant Features

In [49]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.drop(['SalePrice'], axis=1), 
                                                    data.SalePrice, test_size=0.3, 
                                                    random_state=0)

In [50]:
numerical_x_train = x_train[x_train.select_dtypes([np.number]).columns]

In [51]:
from sklearn.feature_selection import VarianceThreshold
vs_constant = VarianceThreshold(threshold=0)

In [52]:
vs_constant.fit(numerical_x_train)

VarianceThreshold(threshold=0)

In [42]:
len(x_train[x_train.select_dtypes([np.number]).columns].columns[constant_filter.get_support()])

35

In [53]:
constant_columns = [column for column in numerical_x_train.columns
                    if column not in numerical_x_train.columns[constant_filter.get_support()]]

In [54]:
len(constant_columns)

0

In [78]:
constant_cat_columns = [column for column in x_train.columns 
                        if (x_train[column].dtype == "O" and len(x_train[column].unique())  == 1 )]

In [79]:
all_constant_columns = constant_cat_columns + constant_columns

In [56]:
x_train.drop(labels=constant_columns, axis=1, inplace=True)
x_test.drop(labels=constant_columns, axis=1, inplace=True)

# Quasi Constant features

In [84]:
def constant_feature_detect(data,threshold=0.98):
    
    quasi_constant_feature = []
    for feature in data.columns:
        predominant = (data[feature].value_counts() / np.float(len(data))).sort_values(ascending=False).values[0]
        if predominant >= threshold:
            quasi_constant_feature.append(feature)   
    return quasi_constant_feature

In [85]:
x_train.drop(labels=constant_feature_detect(x_train), axis=1, inplace=True)
x_test.drop(labels=constant_feature_detect(x_train), axis=1, inplace=True)

# Duplicated Features
In this

In [34]:
train_features_T = x_train.T
train_features_T.head()

,64,682,960,1384,1100,416,1034,853,472,1011,...,1094,599,277,1033,1383,763,835,1216,559,684
Id,65,683,961,1385,1101,417,1035,854,473,1012,...,1095,600,278,1034,1384,764,836,1217,560,685
MSSubClass,60,120,20,50,30,60,30,80,180,90,...,20,160,20,20,30,60,20,90,120,60
MSZoning,RL,RL,RL,RL,RL,RL,RL,RL,RM,RL,...,RL,RM,RL,RL,RL,RL,RL,RM,RL,RL
LotFrontage,70.05,70.05,50,60,60,74,50,70.05,35,75,...,74,24,140,70.05,70.05,82,60,68,70.05,58
LotArea,9375,2887,7207,9060,8400,7844,6305,12095,3675,9825,...,5868,1950,19138,8125,25339,9430,9600,8930,3196,16770


In [68]:
print(train_features_T.duplicated().sum())

0


In [67]:
train_features_T[train_features_T.duplicated()].index.values

array([], dtype=object)